<a href="https://colab.research.google.com/github/ekaratnida/Data_Streaming_and_Realtime_Analytics/blob/main/Week06/pyspark/Pyspark_join_multiple_streams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://mirrors.estointernet.in/apache/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz
!tar -xvf spark-2.4.8-bin-hadoop2.7.tgz
!pip install findspark

In [3]:
!wget "https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-8-assembly_2.11/2.4.8/spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar"

--2022-10-23 14:12:20--  https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-8-assembly_2.11/2.4.8/spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12002039 (11M) [application/java-archive]
Saving to: ‘spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar.1’

spark-streaming-kaf 100%[===================>]  11.45M  --.-KB/s    in 0.07s   

2022-10-23 14:12:20 (164 MB/s) - ‘spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar.1’ saved [12002039/12002039]



In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.8-bin-hadoop2.7"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /content/spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar pyspark-shell'

In [77]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.feature import Normalizer, StandardScaler
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

In [78]:
kafka_topic_name = "my-first-topic"
kafka_topic_name2 = "my-first-topic2"
kafka_bootstrap_servers = 'ec2-13-229-46-113.ap-southeast-1.compute.amazonaws.com:9092'

In [142]:
sc = pyspark.SparkContext()
ssc = StreamingContext(sc,2) # Show results every 2 seconds

kvs1 = KafkaUtils.createDirectStream(ssc, 
                                    [kafka_topic_name], 
                                    {
                        'bootstrap.servers':kafka_bootstrap_servers,
                        'group.id':'test-group',
                        'auto.offset.reset':'smallest'
                        })

kvs2 = KafkaUtils.createDirectStream(ssc, 
                                    [kafka_topic_name2], 
                                    {
                        'bootstrap.servers':kafka_bootstrap_servers,
                        'group.id':'test-group',
                        'auto.offset.reset':'smallest'
                        })

#pairs = lines.map(lambda x: (x.split(" ")[0], x))

kvs1.pprint()
k1 = kvs1.map(lambda x:x[1]).flatMap(lambda x: x.split(",")).map(lambda x: (x.split(":")[0],x.split(":")[1]))
k1.pprint(30)

k2 = kvs2.map(lambda x:x[1]).flatMap(lambda x: x.split(",")).map(lambda x: (x.split(":")[0],x.split(":")[1]))
k2.pprint(30)

k1.fullOuterJoin(k2).pprint(30)

#wc = kvs2.map(lambda x: x[1]).flatMap(lambda line: line.split(' ')).map(lambda word: (word, 1)).reduceByKey(lambda a, b: a+b)
#wc.pprint()
#kvs3 = wc.join(kvs1)
#kvs3.pprint()
'''kvs2 = KafkaUtils.createDirectStream(ssc, 
                                    [kafka_topic_name2], 
                                    {
                        'bootstrap.servers':kafka_bootstrap_servers,
                        'group.id':'test-group1',
                        'auto.offset.reset':'smallest'
                        })

kvs2.pprint()'''

'''step0 = kvs.map(lambda x: x[1])
#print("step0") # print once
step0.pprint()
step1 = step0.flatMap(lambda line: line.split(' '))
step1.pprint()
step2 = step1.map(lambda word: (word, 1))
step2.pprint()
step3 = step2.reduceByKey(lambda a, b: a+b) #.filter(lambda y: y[1]>5)
step3.pprint()'''

ssc.start()
ssc.awaitTerminationOrTimeout(5)
ssc.stop()
sc.stop()

-------------------------------------------
Time: 2022-10-23 15:44:54
-------------------------------------------
('a4066d11-4de2-459c-843d-a6133f464837', ' name:Gal, salary:20000 ')
('f6403d54-a6d1-4219-9cc4-a4d488510a51', ' name:Pok, salary:25000 ')
('c7d9eae6-d873-4d8f-8cf2-4716dc6fbc22', ' name:Vyy, salary:30000 ')

-------------------------------------------
Time: 2022-10-23 15:44:54
-------------------------------------------
(' name', 'Gal')
(' salary', '20000 ')
(' name', 'Pok')
(' salary', '25000 ')
(' name', 'Vyy')
(' salary', '30000 ')

-------------------------------------------
Time: 2022-10-23 15:44:54
-------------------------------------------
(' name', 'Gal')
(' age', '20 ')
(' name', 'Pok')
(' age', '25 ')
(' name', 'Vyy')
(' age', '30 ')

-------------------------------------------
Time: 2022-10-23 15:44:54
-------------------------------------------
(' name', ('Gal', 'Gal'))
(' name', ('Gal', 'Pok'))
(' name', ('Gal', 'Vyy'))
(' name', ('Pok', 'Gal'))
(' name', ('Po

In [138]:
ssc.stop()
sc.stop()